# Bro Log Digestion: FTP

We're taking a look at FTP Bro log. Bro log is a stream of high level entries that correspond to lower level events. A bro log examples are including HTTP request/reply pair, email sent using SMTP, a login over SSH, an FTP command, an so on.

I am using an FTP bro log example generated from MACCDC2012 from [SecRepo](http://www.secrepo.com/) curated by Mike Sconzo with CC-BY license.

First thing to do is to convert `ftp.log` into a readable csv file. I found a useful script to do that from [here](https://github.com/cyberdefendersprogram/MachineLearning)

In [18]:
import csv
import os
dic = {"ftp.log":["ts", "uid", "id_orig_h", "id_orig_p", "id_resp_h", "id_resp_p", "user", "password", "command", "arg","mime_type", "file_size", "reply_code", "reply_msg", "passive", "orig_h", "resp_h", "resp_p", "fuid"]}

path = "/"
for filename in os.listdir(path):
    if filename not in dic:
        continue
    with open(path+"/"+filename.replace("log","csv"), 'w+', encoding='utf-8', newline='') as csvfile:
        w = csv.writer(csvfile, dialect='excel')
        with open(path+"/"+filename, encoding="utf8") as file:
            lines = file.read().split('\n')
            lines=lines[:-1]
            # print(lines)
            files = [dic[filename]]
        for line in lines:
            cells = []
            for item in line.split('\t'):
                if item=="-":
                    cells.append(item.replace("-",""))
                else:
                    cells.append(item)
            files.append(cells)
        w.writerows(files)

There is no print output from the script, but we now got the CSV file of the FTP to ease the `pandas` digestion.

In [20]:
import pandas as pd

df = pd.read_csv("ftp.csv")

#DataFrame with columns
columns = pd.DataFrame(list(df.columns.values[1:]))

#DataFrame with data types
data_types = pd.DataFrame(df.dtypes, columns=['Data Type'])

#DataFrame with Count
data_count = pd.DataFrame(df.count(), columns=['Count'])

#DataFrame with unique values
unique_value_counts = pd.DataFrame(columns=['Unique Values'])
for v in list(df.columns.values):
    unique_value_counts.loc[v] = [df[v].nunique()]

missing_data_counts = pd.DataFrame(df.isnull().sum(), columns=['Missing Values'])
ftp_digestion_report = data_types.join(data_count).join(unique_value_counts).join(missing_data_counts)
print('FTP Digestion Report')
ftp_digestion_report

FTP Digestion Report


,Data Type,Count,Unique Values,Missing Values
ts,float64,5796,2390,0
uid,object,5796,137,0
id_orig_h,object,5796,15,0
id_orig_p,int64,5796,95,0
id_resp_h,object,5796,21,0
id_resp_p,int64,5796,1,0
user,object,5796,4,0
password,object,5745,12,51
command,object,5796,6,0
arg,object,2966,1545,2830


In [21]:
df.head()

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
0,1.331904e+09,CNFo204HUpVHDn1qt2,192.168.203.45,34433,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,219,204).",T,192.168.203.45,192.168.21.101,56268.0,NaN
1,1.331904e+09,CyHkLo2YfhjddpbSVl,192.168.203.45,56158,192.168.21.103,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,103,192,28)",T,192.168.203.45,192.168.21.103,49180.0,NaN
2,1.331904e+09,CotBpLi55vt2fNqm7,192.168.202.96,40138,192.168.28.101,21,<unknown>,NaN,PORT,\x1d\x93!\xf8t\x1ck\xd64\x05\xbb\xbeyu$\x152\x...,NaN,NaN,220.0,ProFTPD 1.3.4rc2 Server (Debian) [::ffff:172.1...,NaN,NaN,NaN,NaN,NaN
3,1.331904e+09,CptK3340W66OKHK3Rd,192.168.202.96,43740,192.168.28.103,21,<unknown>,NaN,PORT,\xbe\xb9wN\x11\xd1\xe1yH\xb8fI\x86\xfdvG\x80\x...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN
4,1.331904e+09,C3NlQu4G9w4W3TGSj7,192.168.204.45,50584,192.168.21.101,21,anonymous,IEUser@,PASV,NaN,NaN,NaN,227.0,"Entering Passive Mode (192,168,21,101,163,245).",T,192.168.204.45,192.168.21.101,41973.0,NaN


In [22]:
#Exclude all unknown user
df.query('user != "<unknown>"').tail()

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
5771,1.332016e+09,CN6ocM3cUJdI5XelNc,192.168.202.102,1090,192.168.23.103,21,anonymous,Cuno,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,530.0,"Not logged in, user account has been disabled",NaN,NaN,NaN,NaN,NaN
5774,1.332016e+09,CLkzRj3FBZD3tx1Uqk,192.168.202.102,1124,192.168.23.103,21,anonymous,Cuno,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,530.0,"Not logged in, user account has been disabled",NaN,NaN,NaN,NaN,NaN
5778,1.332016e+09,C31xPo2cWp7fvsBu7,192.168.202.102,1146,192.168.23.103,21,anonymous,Cuno,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,530.0,"Not logged in, user account has been disabled",NaN,NaN,NaN,NaN,NaN
5783,1.332016e+09,CwXeFh4MZWv1pHqvIe,192.168.202.102,1169,192.168.23.103,21,anonymous,Cuno,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,530.0,"Not logged in, user account has been disabled",NaN,NaN,NaN,NaN,NaN
5788,1.332016e+09,CXP2414L9DvoBYDIZa,192.168.202.102,1193,192.168.23.103,21,anonymous,Cuno,APPE,ftp://192.168.23.103/./\x83\xc7<\xbe\xf0]\xbd\...,NaN,NaN,530.0,"Not logged in, user account has been disabled",NaN,NaN,NaN,NaN,NaN


In [23]:
#Originated from specific IP
df.query("id_orig_h == '192.168.202.96'")

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
2,1.331904e+09,CotBpLi55vt2fNqm7,192.168.202.96,40138,192.168.28.101,21,<unknown>,NaN,PORT,\x1d\x93!\xf8t\x1ck\xd64\x05\xbb\xbeyu$\x152\x...,NaN,NaN,220.0,ProFTPD 1.3.4rc2 Server (Debian) [::ffff:172.1...,NaN,NaN,NaN,NaN,NaN
3,1.331904e+09,CptK3340W66OKHK3Rd,192.168.202.96,43740,192.168.28.103,21,<unknown>,NaN,PORT,\xbe\xb9wN\x11\xd1\xe1yH\xb8fI\x86\xfdvG\x80\x...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN
5483,1.331914e+09,CN2BVs1xAXe8d5fyTe,192.168.202.96,43751,192.168.22.152,21,<unknown>,NaN,PORT,\x1cA\x89\xd6<t4\xb6\xb1\x92'F\xb58\xe3\x12\xd...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN
5495,1.331922e+09,CngrUp21AEmXSytroc,192.168.202.96,53401,192.168.24.103,21,<unknown>,NaN,PORT,z7\x9f4\xbeBv?\xb5\xa8\xb8p|@\x96\x93\x98\x92\...,NaN,NaN,530.0,Please log in with USER and PASS first.,NaN,NaN,NaN,NaN,NaN


In [24]:
#Every Storing file activity
df.query('command == "STOR"').head()

,ts,uid,id_orig_h,id_orig_p,id_resp_h,id_resp_p,user,password,command,arg,mime_type,file_size,reply_code,reply_msg,passive,orig_h,resp_h,resp_p,fuid
14,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/.ftpduBnga4,NaN,NaN,550.0,/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
18,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/.cache/.ftpduBnga4,NaN,NaN,550.0,/.cache/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
22,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/.ssh/.ftpduBnga4,NaN,NaN,550.0,/.ssh/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
26,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/dept/.ftpduBnga4,NaN,NaN,550.0,/dept/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
30,1.331905e+09,Cup8D83JUM166udWb,192.168.202.102,4379,192.168.21.101,21,ftp,password@example.com,STOR,ftp://192.168.21.101/dept/env/.ftpduBnga4,NaN,NaN,550.0,/dept/env/.ftpduBnga4: Operation not permitted,NaN,NaN,NaN,NaN,NaN
